# RAG

## Requirements

In [20]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown

## Dataset

In [21]:
import os
import gdown


os.makedirs('data', exist_ok=True)
gdown.download(url=f"https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI", output='data/IMDB_Crawled.json')

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=35e850dc-2421-46ca-96a1-e4884abf0814
To: /kaggle/working/data/IMDB_Crawled.json
100%|██████████| 292M/292M [00:01<00:00, 279MB/s] 


'data/IMDB_Crawled.json'

## Config

In [22]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [23]:
import pandas as pd


df = pd.read_json('data/IMDB_Crawled.json')

In [24]:
# preprocess your data and only store the needed data as the context window for embedding model is limited
df = df[['title', 'genres', 'rating', 'release_year', 'first_page_summary']]
df = df.dropna()
# df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.columns = ['Title','Genres','Movie Rating','Release Year','Movie Plot']
df['Genres'] = df['Genres'].apply(lambda x: ' '.join(x))
df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [25]:
import pickle

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

from tqdm import tqdm


# Load the CSV
loader = CSVLoader(file_path='data/imdb.csv', metadata_columns=[])
documents = loader.load()

# Embed the documents using the model in a vectorstore
vectorstore = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name=Config.EMBEDDING_MODEL_NAME), distance_strategy=DistanceStrategy.COSINE)

# Save the vectorstore
with open("data/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


load the vectorstore as a retriever.

In [26]:
with open("data/vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": Config.K, 'distance_strategy': DistanceStrategy.COSINE})

## LLM

load the quantized LLM.

In [27]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline


# load the quantization config
bnb_config = BitsAndBytesConfig(
    activation_quantizer="bits",
    weight_quantizer="bits",
    activation_bits=8,
    weight_bits=8,
)

model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

Unused kwargs: ['activation_quantizer', 'weight_quantizer', 'activation_bits', 'weight_bits']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [28]:
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"QUERY:\n{text}")
        return text

query_transform_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("placeholder", "{conversation}"),
    ("system", "Provide a concise search query based on the above conversation."),
    ("assistant", "")
])

def extract_last_assistant(text):
    last_line = text.split('\n')[-1]
    index = last_line.find(":")
    if index != -1:
        extracted_text = last_line[index+1:].strip()
    else:
        extracted_text = last_line.strip()
    if extracted_text.startswith('"') or extracted_text.startswith("'"):
        extracted_text = extracted_text[1:]
    if extracted_text.endswith('"') or extracted_text.endswith("'"):
        extracted_text = extracted_text[:-1]
    return extracted_text.capitalize()

# init the query chain
query_transforming_retriever_chain = (
    query_transform_prompt
    | llm
    | extract_last_assistant
    | LoggerStrOutputParser()
)

initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_core.runnables import RunnablePassthrough


system_prompt = (
    "You are a helpful assistant for question-answering tasks.\n"
    "You MUST choose just one of the movies mentioned below to recommend based on the QUERY.\n"
    "Make sure to write Title, Genres, Movie Rating, Movie Plot and your explanation when you recommend the movie. Also keep your answer concise.\n"
    "Remember MUST choose one of the movies mentioned below."
    "\n\n"
    "Movie choices:"
    "\n"
    "\n"
    "{context}"
    "\n"
    "\n"
    "--------------------"
    "\n"
    "\n"
    "QUERY: {input}"
    "\n"
    "\n"
    "Assistant: Here is my one recommendation movie from the mentioned movies:\n"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),   
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)
    
final_chain = (
    query_transforming_retriever_chain
    | {'input': RunnablePassthrough()}
    | retrieval_chain
)

write the conversation helper class for easier testing.

In [30]:
class Conversation:
    def __init__(self):
        self.messages = []
        self.conversations = []
        self.retrieved_movies = []

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        # concatenate the messages with the roles in the instruction format
        return self.messages
    
    def extract_last_assistant(self, text):
        search_phrase = "Assistant: Here is my one recommendation movie from the mentioned movies:"
        index = text.find(search_phrase)
        if index != -1:
            t = text[index + len(search_phrase):].strip()
        else:
            t = text.strip()
        t = t.strip('"\'')
        return t.capitalize()

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        # invoke the chain
        response = final_chain.invoke({'conversation': messages})
        self.conversations.append(response['answer'])
        self.retrieved_movies.append(response['context'])
        assistant_response = self.extract_last_assistant(response['answer'])
        self.add_assistant_message(assistant_response)
        return '\n' + 'RESPONSE:' + '\n' + assistant_response

## Test

talk with the RAG to see how good it performs.

In [31]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print(A)

QUERY:
Recommend a stylish and intense gangster movie with a charismatic protagonist and a complex plot.

RESPONSE:
Title: scarface (1983)
genres: crime drama
movie rating: 8.3
release year: 1983
movie plot: in 1980 miami, a determined cuban immigrant takes over a drug cartel and succumbs to greed.

explanation: scarface is a stylish and intense gangster movie that fits your criteria perfectly. the movie's protagonist, tony montana, played by al pacino, is charismatic, complex, and delivers an iconic performance. the plot is intricate, with twists and turns that keep you engaged till the end. the movie's cinematography, soundtrack, and editing are also top-notch, making it a must-watch for any gangster movie enthusiast.


In [32]:
A = c.chat('give me a newer one')
print(A)

QUERY:
Recommend a stylish and intense gangster movie released after 2010 with a complex protagonist and a gripping plot.

RESPONSE:
Title: the irishman
genres: biographical crime drama
movie rating: 7.5
release year: 2019
movie plot: a mob hitman recalls his possible involvement with the slaying of jimmy hoffa.

the irishman is a stylish and intense gangster movie released in 2019, with a complex protagonist and a gripping plot. the movie follows the life of frank sheeran, a hitman for the bufalino crime family, as he recalls his possible involvement in the slaying of jimmy hoffa. the movie is based on the book "i heard you paint houses" by charles brandt, and it features an all-star cast, including robert de niro, al pacino, and joe pesci. the movie's direction by martin scorsese is masterful, and the cinematography by rodrigo prieto is stunning. the irishman is a must-watch for any fan of the gangster genre.


In [33]:
c = Conversation()
A = c.chat('give me a sci-fi movie about interplanetary traveling')
print(A)

QUERY:
Recommend a sci-fi movie with a focus on interplanetary travel, featuring advanced technology and thrilling action sequences.

RESPONSE:
Title: interstellar
genres: adventure drama sci-fi
movie rating: 8.7
release year: 2014

explanation: interstellar is the perfect choice for your query as it features interplanetary travel, advanced technology, and thrilling action sequences. the movie follows a team of researchers as they embark on a dangerous mission to find a new habitable planet for humanity, which is facing a catastrophic event on earth. the movie's visual effects and the use of advanced technology, such as the tars robots, make it a must-watch for sci-fi enthusiasts. the movie's action sequences, including the zero-gravity fight scene, will keep you on the edge of your seat. overall, interstellar is a masterpiece that combines science, drama, and action in a captivating way.


In [34]:
A = c.chat('give me an older one')
print(A)

QUERY:
Sci-fi movie about interplanetary travel from the 90s

RESPONSE:
Title: stargate
genres: action adventure sci-fi
movie rating: 7.0
release year: 1994

explanation: "stargate" is a sci-fi movie about interplanetary travel from the 90s that fits the query. it follows the story of an interstellar teleportation device, found in egypt, that leads to a planet with humans resembling ancient egyptians who worship the god ra. the movie's special effects and action sequences make it an exciting watch, and it's a classic sci-fi movie from the 90s.


In [35]:
c = Conversation()
A = c.chat('give me a movie about magic and wizardry')
print(A)

QUERY:
Recommend a movie with elements of magic and wizardry for me to watch.

RESPONSE:
Title: harry potter and the sorcerer's stone
genres: adventure family fantasy
movie rating: 7.6
release year: 2001
movie plot: an orphaned boy enrolls in a school of wizardry, where he learns the truth about himself, his family and the terrible evil that haunts the magical world.

explanation: this movie is a classic in the genre of magic and wizardry. it follows the story of harry potter, a young boy who discovers he is a wizard and is invited to attend hogwarts school of witchcraft and wizardry. the movie is filled with magical elements, from spells and potions to creatures like hagrid's giant spider and dumbledore's phoenix. it's a great choice for anyone who loves the world of harry potter or is a fan of fantasy and magic.


In [36]:
A = c.chat('give me a similar movie to this')
print(A)

QUERY:
Recommend a movie similar to harry potter and the sorcerer's stone in terms of genre and rating, with a focus on magic and wizardry.

RESPONSE:
Title: harry potter and the chamber of secrets
genres: adventure family fantasy
movie rating: 7.4
release year: 2002
movie plot: an ancient prophecy seems to be coming true when a mysterious presence begins stalking the corridors of a school of magic and leaving its victims paralyzed.

this movie is similar to harry potter and the sorcerer's stone in terms of genre and rating, with a focus on magic and wizardry. it's the second movie in the harry potter series, and it continues the adventures of harry potter, ron weasley, and hermione granger as they uncover the secrets of the chamber of secrets, a mysterious and dangerous place at hogwarts school of witchcraft and wizardry. the movie features magical creatures, spells, and challenges that will appeal to fans of the first movie and the harry potter books.


In [37]:
A = c.chat('give me a more recent harry potter movie')
print(A)

QUERY:
Harry potter movies released after 2010

RESPONSE:
Title: fantastic beasts: the secrets of dumbledore
genres: adventure family fantasy
movie rating: 6.2
release year: 2022
movie plot: professor albus dumbledore must assist newt scamander and his partners as grindelwald begins to lead an army to eliminate all muggles.

explanation: while the first harry potter movie released after 2010 is "harry potter and the deathly hallows: part 2," which is a continuation of the previous movies, "fantastic beasts: the secrets of dumbledore" is a standalone movie set in the wizarding world, featuring a new storyline and characters. it's a great choice for those who have already watched the harry potter movies and are looking for a fresh experience.


In [38]:
c = Conversation()
A = c.chat('give me a movie based on books written by frank herbert')
print(A)

QUERY:
Recommend a movie adaptation of a frank herbert novel

RESPONSE:
Title: dune (2021)
genres: action adventure drama sci-fi
movie rating: 8.0
release year: 2021
movie plot: a noble family becomes embroiled in a war for control over the galaxy's most valuable asset while its heir becomes troubled by visions of a dark future.

explanation: "dune" (2021) is a highly anticipated movie adaptation of frank herbert's classic science fiction novel. directed by denis villeneuve, this movie boasts an impressive cast including timothée chalamet, oscar isaac, and zendaya. the movie's stunning visuals and immersive world-building make it a must-watch for fans of the original novel and newcomers alike. with a high rating and positive reviews, "dune" (2021) is the clear choice for a movie adaptation of a frank herbert novel.
